# Large Language Models (LLMs) with Langchain

Things that we can do with Langchain:

- LLMs: Get predictions from language model
- Prompt Templates: manage prompts for LLMs
- Chains: combine LLMs and prompts in multi-step workflows
- Agents: dynamically call chains based on user input
- Memory: add state to Chain and Agents


> To better understand the modules/variables, suffix the variables with a question mark, e.g. OpenAI? or press CMD + I and click on the variables to display contextual help.

## LLMs: Get predictions from a language model

Let's build a service that generates a company name based on what the company makes.

In [8]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
text = "What would be a good company name for a startup that specializes in stable diffusion?"

print(llm(text))



Stable Diffusion Solutions


## Prompt Templates: manage prompts for LLMs

When using LLMs, we usually work with a specific prompt. Instead of constructing it everytime, we can use prompt template with variables substitution.

In [9]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["tech"],
    template="What would be a good company name for a startup that specializes in {tech}?",
)
print(prompt.format(tech="deep learning"))

What would be a good company name for a startup that specializes in deep learning?


## Chains: combine LLMs and prompts in multi-step workflows

PromptTemplate and LLMs are primitives by themselves. Applications usually are a combination of primitives.

A chain in LangChain is made up of links, which can be either primitives like LLMs or other chains.

In [11]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["tech"],
    template="What would be a good company name for a startup that specializes in {tech}?",
)

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("deep learning"))



DeepLearnTech Solutions


## Agents: Dynamically Call Chains based on user input

Unlike chains, agents do not run in a predetermined order.

Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

Some important concepts when loading agents:

- **Tool**: a function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool currently is `fn(str): -> str`, a function that is expected to have a string as an input, with a string as an output
- **LLM**: The language model powering an agent
- **Agent**: The agent to use.

In [12]:
# -qq only prints out error
!pip install -qq google-search-results

In [21]:
from langchain.agents import initialize_agent, load_tools
from langchain.llms import OpenAI

# First, let's load the language model we are going to use to control the agent.
# We set the temperature, because we do not want our agent to be too creative.
llm = OpenAI(temperature=0)

# Next, we load some tools to use.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run(
    "What was the high temperature in Singapore yesterday in Celcius? What is the number raised to the power of 0.5?"
)



> Entering new AgentExecutor chain...
 I need to find out the temperature in Singapore and then calculate the power of 0.5
Action: Search
Action Input: "High temperature in Singapore yesterday"
Observation: High & Low Weather Summary for the Past Weeks ; 82 °F · 78% ; * Reported Mar 26 5:00 pm — Apr 10 5:35 pm, Singapore.
Thought: I need to convert the temperature from Fahrenheit to Celcius
Action: Calculator
Action Input: 82°F to Celcius
Observation: Answer: 27.77777777777778

Thought: I need to calculate the power of 0.5
Action: Calculator
Action Input: 27.77777777777778^0.5
Observation: Answer: 5.270462766947299

Thought: I now know the final answer
Final Answer: The high temperature in Singapore yesterday in Celcius was 27.77777777777778 and the number raised to the power of 0.5 is 5.270462766947299.

> Finished chain.


'The high temperature in Singapore yesterday in Celcius was 27.77777777777778 and the number raised to the power of 0.5 is 5.270462766947299.'

## Memory: Add state to Chains and Agents

Chains and Agents can be stateless or stateful. Stateful Chain or Agent have some concepts of "memory", so that it may remember information about its previous interaction.

Some relevant application is a chatbot - you want it to remember the previous message so that it can use the context from that to have a better conversation.

In [22]:
from langchain import ConversationChain, OpenAI

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)
conversation.predict(input="Hi there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there
AI:

> Finished chain.


' Hi there! How can I help you?'

In [23]:
conversation.predict(input='Just wanna ask your thoughts on AI')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi there! How can I help you?
Human: Just wanna ask your thoughts on AI
AI:

> Finished chain.


" AI is an incredibly powerful technology that has the potential to revolutionize many aspects of our lives. It can be used to automate mundane tasks, improve decision-making, and even create new products and services. AI can also be used to improve healthcare, education, and transportation. I'm excited to see what the future holds for AI!"